In [15]:
import os
import cv2
import numpy as np
import random
from google.colab.patches import cv2_imshow
from keras.preprocessing import image

In [2]:
categories=['with_mask','without_mask']

In [7]:
data=[]
for category in categories:
  path=os.path.join("/content/drive/MyDrive/Deep learning/data",category)

  label=categories.index(category)
  for file in os.listdir(path):
    img_path=os.path.join(path,file)
    img=cv2.imread(img_path)
    img=cv2.resize(img,(224,224))
    
    data.append([img,label])

In [8]:
len(data)

1376

In [11]:
random.shuffle(data)

In [12]:
x=[]
y=[]
for feature,label in data:
  x.append(feature)
  y.append(label)

In [16]:
x=np.array(x)
y=np.array(y)

In [17]:
x.shape,y.shape

((1376, 224, 224, 3), (1376,))

In [18]:
x=x/255

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [20]:
X_train.shape,y_train.shape

((1100, 224, 224, 3), (1100,))

In [31]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense

In [24]:
vgg=VGG16()

553467096/553467096 [==============================] - 7s 0us/step


In [25]:
model=Sequential()

In [27]:
for layer in vgg.layers[:-1]:
  model.add(layer)

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [29]:
for layer in model.layers:
  layer.trainable=False

In [32]:
model.add(Dense(1,activation='sigmoid'))

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [34]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [35]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
35/35 [==============================] - 22s 311ms/step - loss: 0.4903 - accuracy: 0.8118 - val_loss: 0.3527 - val_accuracy: 0.8986
Epoch 2/5
35/35 [==============================] - 7s 201ms/step - loss: 0.2902 - accuracy: 0.9236 - val_loss: 0.2858 - val_accuracy: 0.8949
Epoch 3/5
35/35 [==============================] - 7s 202ms/step - loss: 0.2262 - accuracy: 0.9309 - val_loss: 0.2017 - val_accuracy: 0.9275
Epoch 4/5
35/35 [==============================] - 7s 203ms/step - loss: 0.1915 - accuracy: 0.9382 - val_loss: 0.1812 - val_accuracy: 0.9457
Epoch 5/5
35/35 [==============================] - 7s 205ms/step - loss: 0.1550 - accuracy: 0.9582 - val_loss: 0.1568 - val_accuracy: 0.9457


In [36]:
model.save("/content/drive/MyDrive/Colab Notebooks/model.h5")

In [ ]:
import numpy as np
import pandas as pd
import cv2
import tensorflow
from tensorflow.keras.models import load_model

In [ ]:
model = load_model("model_new.h5")
def detect_face(img):
    predict_y = (model.predict(img.reshape(1,224,224,3)) > 0.5).astype("int32")
    return predict_y[0][0]


In [63]:



# sample=cv2.imread("without.jpg")
# sample=cv2.resize(sample,(224,224))





cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame=cv2.resize(frame,(224,244))
    y_pred=detect_face(frame)

    print(y_pred)

    cv2.imshow("Window", frame)

    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
cv2.destroyAllWindows()

ValueError: cannot reshape array of size 163968 into shape (1,224,224,3)